# Quantitative trading 

This project implements a machine learning-based trading strategy for Palantir (PLTR) stock using two years of hourly price data. The workflow includes data acquisition, cleaning, and feature engineering with technical indicators such as EMAs, RSI, and ATR.

A target variable is created based on EMA crossovers to indicate buy/sell signals. The dataset is scaled and dimensionality reduced using PCA before training multiple classifiers: Gradient Boosting, Support Vector Machine, Logistic Regression, and LSTM.

Model predictions are backtested in a simulated trading environment considering transaction costs and slippage. Entry and exit points are visualized on cumulative return plots. Performance evaluation includes metrics such as annualized returns, Sharpe and Sortino ratios, and trade-level profit and loss.

This approach provides a comparative analysis of traditional machine learning and deep learning methods applied to algorithmic trading.

In [1]:
# import libraries
import pandas as pd
import yfinance as yf
import hvplot.pandas
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

import warnings
warnings.filterwarnings('ignore')

## Step 1: Data Downloading and Basic Preprocessing
Download hourly historical price data for the PLTR stock over the last 2 years using the Yahoo Finance API.

Remove multi-level column headers for simplified dataframe structure.

Eliminate redundant column name metadata for clarity.

Round key price and volume columns to two decimal places for numerical consistency.

Convert the dataframe index to the US Eastern timezone to match market time.

Display the processed data and visualize the hourly closing price to gain an initial insight into the price trends.

In [2]:
# download the historical prices of pltr
pltr_df = yf.download(tickers = 'pltr', period='2Y', interval = '1h')
pltr_df

# Remove multilevels of the column headers
pltr_df = pltr_df.droplevel(level = 1, axis = 1)

# Remove the name 'Price' from the headers
pltr_df.columns.name = None

pltr_df

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
Datetime,,,,,
2023-07-03 13:30:00+00:00,15.319300,15.590000,15.220000,15.490000,12379571
2023-07-03 14:30:00+00:00,15.405000,15.420000,15.190000,15.310000,5627704
2023-07-03 15:30:00+00:00,15.470000,15.470000,15.340000,15.407100,3471561
2023-07-05 13:30:00+00:00,15.425000,15.570000,15.230000,15.436700,9888941
2023-07-05 14:30:00+00:00,15.345000,15.580000,15.330000,15.430000,6028562
...,...,...,...,...,...
2025-07-02 15:30:00+00:00,132.240005,132.320007,130.960007,131.490005,5249743
2025-07-02 16:30:00+00:00,132.660004,133.369995,132.110001,132.220001,6213849
2025-07-02 17:30:00+00:00,132.509995,133.300293,132.179993,132.670700,5675254


In [3]:
# Round the values of the dataframe to 2 decimal points
signals_df = round(pltr_df[['Close', 'High', 'Low', 'Open', 'Volume']], 2)

# Change the index to the required timezone
signals_df.index = signals_df.index.tz_convert('US/Eastern')

# Display the data
signals_df

,Close,High,Low,Open,Volume
Datetime,,,,,
2023-07-03 09:30:00-04:00,15.32,15.59,15.22,15.49,12379571
2023-07-03 10:30:00-04:00,15.40,15.42,15.19,15.31,5627704
2023-07-03 11:30:00-04:00,15.47,15.47,15.34,15.41,3471561
2023-07-05 09:30:00-04:00,15.43,15.57,15.23,15.44,9888941
2023-07-05 10:30:00-04:00,15.35,15.58,15.33,15.43,6028562
...,...,...,...,...,...
2025-07-02 11:30:00-04:00,132.24,132.32,130.96,131.49,5249743
2025-07-02 12:30:00-04:00,132.66,133.37,132.11,132.22,6213849
2025-07-02 13:30:00-04:00,132.51,133.30,132.18,132.67,5675254


In [78]:
# visulise the closing price
signals_df['Close'].hvplot(title = 'Hourly Closing Price of PLTR')

:Curve   [Datetime]   (Close)

## Step 2: Feature Engineering and Signal Generation
Define short-term (20 periods) and long-term (50 periods) windows for moving averages.

Calculate Exponential Moving Averages (EMA) of the closing price for both short and long windows to capture price trends.

Compute the Relative Strength Index (RSI) over a 14-period window to quantify momentum and identify overbought/oversold conditions.

Calculate the 14-period Average True Range (ATR) to measure market volatility based on price ranges.

Clean the dataset by removing rows with missing values introduced by rolling calculations.

Generate the binary target signal based on the relationship between the short-term and long-term EMA (1 if short EMA > long EMA, else 0), serving as the predictive label for subsequent modeling.

In [5]:
# Set the long and short windows
short_window = 20
long_window = 50

# Obtain the Exponential Moving Average of the Close prices with short and long windows
signals_df['EMA20_Close'] =  round(signals_df['Close'].ewm(span = short_window).mean(), 2)
signals_df['EMA50_Close'] =  round(signals_df['Close'].ewm(span = long_window).mean(), 2)

# Calculate Relative Strength Index (RSI)
delta = signals_df['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
signals_df['RSI'] = 100 - (100 / (1 + rs))

# Calculate the 14-day Average True Range (ATR) for volatility
high_low = signals_df['High'] - signals_df['Low']
high_close = (signals_df['High'] - signals_df['Close'].shift()).abs()
low_close = (signals_df['Low'] - signals_df['Close'].shift()).abs()
tr = pd.concat([high_low, high_close, low_close], axis=1)
signals_df['ATR'] = tr.max(axis=1).rolling(window=14).mean()

# Drop missing values
signals_df.dropna(inplace=True)

# view data
signals_df.head()

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR
Datetime,,,,,,,,,
2023-07-06 12:30:00-04:00,15.06,15.14,14.95,14.97,4736592,15.27,15.31,42.857143,0.274286
2023-07-06 13:30:00-04:00,15.13,15.16,15.01,15.05,5024314,15.25,15.29,44.973545,0.258571
2023-07-06 14:30:00-04:00,15.17,15.19,15.09,15.14,4433036,15.24,15.28,43.783784,0.249286
2023-07-06 15:30:00-04:00,15.13,15.18,15.10,15.17,4850850,15.23,15.27,40.659341,0.245714
2023-07-07 09:30:00-04:00,15.63,15.90,15.22,15.23,17897837,15.27,15.30,54.385965,0.276429


In [6]:
# Create signals
signals_df['Target'] = (signals_df['EMA20_Close'] > signals_df['EMA50_Close']).astype(int)

# Display data
signals_df.head()

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Target
Datetime,,,,,,,,,,
2023-07-06 12:30:00-04:00,15.06,15.14,14.95,14.97,4736592,15.27,15.31,42.857143,0.274286,0
2023-07-06 13:30:00-04:00,15.13,15.16,15.01,15.05,5024314,15.25,15.29,44.973545,0.258571,0
2023-07-06 14:30:00-04:00,15.17,15.19,15.09,15.14,4433036,15.24,15.28,43.783784,0.249286,0
2023-07-06 15:30:00-04:00,15.13,15.18,15.10,15.17,4850850,15.23,15.27,40.659341,0.245714,0
2023-07-07 09:30:00-04:00,15.63,15.90,15.22,15.23,17897837,15.27,15.30,54.385965,0.276429,0


## Step 3: Machine Learning - Data Preparation, Model Training, and Evaluation

- Separate features (`X`) and target (`y`) variables for supervised learning.
- Perform a chronological train-test split (70% train, 30% test) without shuffling to maintain time series order.
- Apply standard scaling (`StandardScaler`) to normalize feature distributions.
- Use Principal Component Analysis (PCA) to reduce dimensionality to 5 components, retaining most variance.
- Check class balance in the training target labels.
- Train and evaluate the following classification models on PCA-transformed data:
  - Gradient Boosting Classifier (300 estimators, max depth 3)
  - Support Vector Machine (linear kernel)
  - Logistic Regression (`liblinear` solver)
- Generate classification reports for each model, detailing precision, recall, F1-score, and support.
- Prepare input data for Long Short-Term Memory (LSTM) model by reshaping into 3D tensors `[samples, timesteps, features]`.
- Build a stacked LSTM network with four layers and dropout for regularization.
- Compile and train the LSTM model over 50 epochs with batch size 32.
- Generate predictions from the LSTM and apply thresholding to obtain binary signals.
- Output classification metrics for LSTM predictions to compare model performance.


In [7]:
# Define the datasets X and y for machine learing
X = signals_df.drop(columns = ['Target'], axis = 1)
y = signals_df['Target']


# Split the data into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, shuffle = False, random_state = 2)

In [8]:
# Scale the data using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Conduct PCA transformation
pca = PCA(n_components= 5)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [9]:
# Check if the training data label classes are balanced 
y_train.value_counts()

Target
1    1395
0    1041
Name: count, dtype: int64

### Gradienct Boosting Classifier

In [10]:
# initiate the model
gb_model = GradientBoostingClassifier(n_estimators = 300,
                                      max_depth = 3,
                                      learning_rate = .1,
                                      random_state = 1)

# fit the model with the training data
gb_model.fit(X_train_pca, y_train)


# Obtain the prediction with the model
y_pred_gb = gb_model.predict(X_test_pca)

# Generate the Classification report
print('Gradient Boositing Classifier',classification_report(y_test, y_pred_gb))

Gradient Boositing Classifier               precision    recall  f1-score   support

           0       0.23      0.10      0.13       315
           1       0.69      0.86      0.76       730

    accuracy                           0.63      1045
   macro avg       0.46      0.48      0.45      1045
weighted avg       0.55      0.63      0.57      1045



### Supported Vector Machine

In [11]:
# Initialize and fit the data to SVC
svm_model = SVC(kernel = 'linear', gamma = 'auto', C=1.0)
svm_model.fit(X_train_pca, y_train)


# make predictions with the model
y_pred_svm = svm_model.predict(X_test_pca)

# Obtain the classification report of predictions against the test data
print('Supported Vector Machine \n',classification_report(y_test, y_pred_svm))

Supported Vector Machine 
               precision    recall  f1-score   support

           0       0.92      0.32      0.47       315
           1       0.77      0.99      0.87       730

    accuracy                           0.79      1045
   macro avg       0.84      0.65      0.67      1045
weighted avg       0.81      0.79      0.75      1045



### Logistic Regression

In [12]:
# Initiate and fit the data to logistic regression model
lr_model = LogisticRegression(solver = 'liblinear')
lr_model.fit(X_train_pca, y_train)

# Make predictions with the model
y_pred_lr = lr_model.predict(X_test_pca)

# Obtain the classification report of the predictions against the test data
print('Logistic Regression\n',classification_report(y_test, y_pred_lr))

Logistic Regression
               precision    recall  f1-score   support

           0       0.95      0.17      0.29       315
           1       0.74      1.00      0.85       730

    accuracy                           0.75      1045
   macro avg       0.84      0.59      0.57      1045
weighted avg       0.80      0.75      0.68      1045



### LSTM

In [13]:
# Reshape input to 3D [samples, time_steps, features] to appropriately fir the data into the LSTM model
X_train_reshaped = np.reshape(X_train_pca, (X_train_pca.shape[0], X_train_pca.shape[1], 1))

# Initialized LSTM Classifier
lstm = Sequential()

# Creating the first hidden layer
lstm.add(LSTM(units = 70, return_sequences = True, input_shape = (X_train_pca.shape[1],1)))
lstm.add(Dropout(0.2))

# Creating the second hidden layer
lstm.add(LSTM(units = 60, return_sequences = True, activation = 'relu'))
lstm.add(Dropout(0.1))

# Creating the third hidden layer
lstm.add(LSTM(units = 60, return_sequences = True, activation = 'relu'))
lstm.add(Dropout(0.1))

# Creating the fourth hidden layer
lstm.add(LSTM(units = 60, activation = 'relu'))
lstm.add(Dropout(0.1))

# Creating the output layer to the model 
lstm.add(Dense(units = 1, activation= 'sigmoid'))

In [14]:
# Obtain the LSTM model symmary
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 70)             20160     
                                                                 
 dropout (Dropout)           (None, 5, 70)             0         
                                                                 
 lstm_1 (LSTM)               (None, 5, 60)             31440     
                                                                 
 dropout_1 (Dropout)         (None, 5, 60)             0         
                                                                 
 lstm_2 (LSTM)               (None, 5, 60)             29040     
                                                                 
 dropout_2 (Dropout)         (None, 5, 60)             0         
                                                                 
 lstm_3 (LSTM)               (None, 60)                2

In [15]:
# Compile the model
lstm.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [16]:
# Fit the model
lstm.fit(X_train_reshaped, y_train, epochs=50, batch_size=32)

Epoch 1/50
77/77 [==============================] - 6s 29ms/step - loss: 0.2193 - accuracy: 0.6363
Epoch 2/50
77/77 [==============================] - 2s 30ms/step - loss: 0.1982 - accuracy: 0.6872
Epoch 3/50
77/77 [==============================] - 2s 31ms/step - loss: 0.1977 - accuracy: 0.6892
Epoch 4/50
77/77 [==============================] - 2s 31ms/step - loss: 0.1995 - accuracy: 0.6839
Epoch 5/50
77/77 [==============================] - 2s 28ms/step - loss: 0.1960 - accuracy: 0.6884
Epoch 6/50
77/77 [==============================] - 2s 31ms/step - loss: 0.1970 - accuracy: 0.6909
Epoch 7/50
77/77 [==============================] - 2s 32ms/step - loss: 0.1957 - accuracy: 0.6946
Epoch 8/50
77/77 [==============================] - 2s 32ms/step - loss: 0.1964 - accuracy: 0.6942
Epoch 9/50
77/77 [==============================] - 2s 31ms/step - loss: 0.1952 - accuracy: 0.6897
Epoch 10/50
77/77 [==============================] - 2s 23ms/step - loss: 0.1933 - accuracy: 0.7044
Epoch 11/

In [17]:
# Make predictions
y_pred = lstm.predict(X_test_pca)

# Obtain the labels in the required format
y_pred_lstm = np.where(y_pred > y_pred.mean(), 1, 0).ravel()

33/33 [==============================] - 1s 8ms/step


In [18]:
# Generate the classification report of the predictions against the test set
print('LSTM',classification_report(y_test, y_pred_lstm))

LSTM               precision    recall  f1-score   support

           0       0.36      0.36      0.36       315
           1       0.72      0.72      0.72       730

    accuracy                           0.61      1045
   macro avg       0.54      0.54      0.54      1045
weighted avg       0.61      0.61      0.61      1045



## Step 4: Backtesting Model Predictions and Analyzing Returns

- Define `backtest_df` function to simulate portfolio performance based on model predictions:
  - Shift predictions forward by one time step to apply signals in the next period.
  - Initialize capital, position, and simulate trading decisions:
    - Buy when model predicts long signal and no current position.
    - Sell when model predicts no position and a current holding exists.
  - Account for transaction costs (default 0.15%) and slippage (default 0.05%) in trade execution.
  - Track portfolio value over time as cash plus current position value.
  - Return a DataFrame containing model predictions and portfolio total value indexed by time.

- Define `cumulative_returns` function to calculate and visualize portfolio performance:
  - Merge portfolio values with feature data for validation.
  - Compute hourly portfolio returns and cumulative returns as compounded growth.
  - Return a DataFrame including cumulative returns for performance analysis.

- Apply the backtesting functions to predictions from each model (Gradient Boosting, SVM, Logistic Regression, LSTM):
  - Generate backtested portfolio values and cumulative returns.
  - Plot cumulative returns over time to visually compare model-driven strategies.

This step allows evaluation of how each model’s trading signals would have performed financially, incorporating realistic trading frictions.

In [56]:
# Create function to accept the validation input data and model predictions
# Generate a data frome through the function with the corresponding returns based on the model predictions entered
def backtest_df(df, pred, transaction_cost = 0.0015, slippage_rate = 0.0005):
    
    # shift predictions to the next hour so that the model predictions are correctly applied
    pred = np.roll(pred, 1)
    
    # initiate first prediction to zero
    pred[0] = 0
     
    # Initiate the other required variables for backtesting
    capital, position = 100000, 0
    portfolio = []
    
    
    for i in range(len(df)):
        close_price = df.iloc[i]['Close']
        
        # Algorithm to purchase if there is no position and the predicted signal is to buy
        if pred[i] == 1 and position == 0:
            
            position = capital / (close_price * (1 + slippage_rate)) # Simulating involvement of slippage rate 
            
            # Transaction cost deduction
            cost_of_trade = capital * transaction_cost
            capital = 0 # all of capital invested
            capital -= cost_of_trade
        
        # Algorithm to sell if a position is held and the predicted signal is to sell
        elif pred[i] == 0 and position != 0:
            capital = position * close_price * (1 - slippage_rate)
            # sell all position
            cost_of_trade = capital * transaction_cost
            capital -= cost_of_trade
            position = 0
        
        # Calculate portfolio_value and append to the portfolio list
        portfolio_value = capital + position * close_price
        portfolio.append(portfolio_value)
        
    return pd.DataFrame({'Model_Predictions': pred, 'Portfolio_Total': portfolio}, index=df.index)

In [57]:
# Function to generate dataframe with the validation input and cumulative returns and its plot based on the model predictions 
def cumualtive_returns(portfolio_df, X_test):
    
    # Merge
    merged_df = pd.concat([X_test, portfolio_df], axis = 1)
    
    # Obtain the Daily returns of the portfolio and clean the data
    merged_df['Portfolio_Returns'] = merged_df['Portfolio_Total'].pct_change()
    merged_df.dropna(inplace = True)
    
    # Calculate cumualtive returns at every hour
    merged_df['Cumulative_Returns'] = (1 + merged_df['Portfolio_Returns']).cumprod() - 1
    
    return merged_df

##### Gradient Boosting Classifier

In [58]:
# Conduct backtest with the function created
gb_backtest = backtest_df(X_test, y_pred_gb)

# Obtain the cumulateive returns the backtesting algorithm generates
gb_backtest_cumulative = cumualtive_returns(gb_backtest, X_test)
gb_backtest_cumulative

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Model_Predictions,Portfolio_Total,Portfolio_Returns,Cumulative_Returns
Datetime,,,,,,,,,,,,,
2024-11-21 14:30:00-05:00,61.58,61.74,61.30,61.59,4064551,61.75,60.79,45.707376,1.046429,1,99800.024988,-0.002000,-0.002000
2024-11-21 15:30:00-05:00,61.35,61.72,61.24,61.57,3388214,61.71,60.81,39.641944,1.027143,1,99426.713754,-0.003741,-0.005733
2024-11-22 09:30:00-05:00,62.21,63.04,61.37,61.60,15777196,61.76,60.87,63.665595,0.925000,1,100822.573148,0.014039,0.008226
2024-11-22 10:30:00-05:00,63.34,63.47,62.14,62.21,11239611,61.91,60.97,68.870523,0.956429,1,102656.667468,0.018191,0.026567
2024-11-22 11:30:00-05:00,64.04,64.38,63.00,63.35,12237053,62.11,61.09,68.347339,0.975000,1,103792.832092,0.011068,0.037928
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 11:30:00-04:00,132.24,132.32,130.96,131.49,5249743,133.70,136.57,28.648649,1.957143,0,182041.792454,0.005200,0.820418
2025-07-02 12:30:00-04:00,132.66,133.37,132.11,132.22,6213849,133.60,136.42,32.770745,1.904286,0,182041.792454,0.000000,0.820418
2025-07-02 13:30:00-04:00,132.51,133.30,132.18,132.67,5675254,133.50,136.26,32.002801,1.927143,0,182041.792454,0.000000,0.820418


In [59]:
# Plot the cumulative returns of the model from the backtesting
gb_backtest_cumulative['Cumulative_Returns'].hvplot()

:Curve   [Datetime]   (Cumulative_Returns)

##### Supported Vector Machine

In [60]:
# Conduct backtest with the function created
svm_backtest = backtest_df(X_test, y_pred_svm)

# Obtain the cumulateive returns the backtesting algorithm generates
svm_backtest_cumulative = cumualtive_returns(svm_backtest, X_test)
svm_backtest_cumulative

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Model_Predictions,Portfolio_Total,Portfolio_Returns,Cumulative_Returns
Datetime,,,,,,,,,,,,,
2024-11-21 14:30:00-05:00,61.58,61.74,61.30,61.59,4064551,61.75,60.79,45.707376,1.046429,1,99800.024988,-0.002000,-0.002000
2024-11-21 15:30:00-05:00,61.35,61.72,61.24,61.57,3388214,61.71,60.81,39.641944,1.027143,1,99426.713754,-0.003741,-0.005733
2024-11-22 09:30:00-05:00,62.21,63.04,61.37,61.60,15777196,61.76,60.87,63.665595,0.925000,1,100822.573148,0.014039,0.008226
2024-11-22 10:30:00-05:00,63.34,63.47,62.14,62.21,11239611,61.91,60.97,68.870523,0.956429,1,102656.667468,0.018191,0.026567
2024-11-22 11:30:00-05:00,64.04,64.38,63.00,63.35,12237053,62.11,61.09,68.347339,0.975000,1,103792.832092,0.011068,0.037928
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 11:30:00-04:00,132.24,132.32,130.96,131.49,5249743,133.70,136.57,28.648649,1.957143,1,198980.280172,0.005636,0.989803
2025-07-02 12:30:00-04:00,132.66,133.37,132.11,132.22,6213849,133.60,136.42,32.770745,1.904286,1,199613.194740,0.003181,0.996132
2025-07-02 13:30:00-04:00,132.51,133.30,132.18,132.67,5675254,133.50,136.26,32.002801,1.927143,1,199387.153823,-0.001132,0.993872


In [61]:
# Plot the cumulative returns of the model from the backtesting
svm_backtest_cumulative['Cumulative_Returns'].hvplot()

:Curve   [Datetime]   (Cumulative_Returns)

##### Logistic Regression

In [62]:
# Conduct backtest with the function created
lr_backtest = backtest_df(X_test, y_pred_lr)

# Obtain the cumulateive returns the backtesting algorithm generates
lr_backtest_cumulative = cumualtive_returns(lr_backtest, X_test)
lr_backtest_cumulative

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Model_Predictions,Portfolio_Total,Portfolio_Returns,Cumulative_Returns
Datetime,,,,,,,,,,,,,
2024-11-21 14:30:00-05:00,61.58,61.74,61.30,61.59,4064551,61.75,60.79,45.707376,1.046429,1,99800.024988,-0.002000,-0.002000
2024-11-21 15:30:00-05:00,61.35,61.72,61.24,61.57,3388214,61.71,60.81,39.641944,1.027143,1,99426.713754,-0.003741,-0.005733
2024-11-22 09:30:00-05:00,62.21,63.04,61.37,61.60,15777196,61.76,60.87,63.665595,0.925000,1,100822.573148,0.014039,0.008226
2024-11-22 10:30:00-05:00,63.34,63.47,62.14,62.21,11239611,61.91,60.97,68.870523,0.956429,1,102656.667468,0.018191,0.026567
2024-11-22 11:30:00-05:00,64.04,64.38,63.00,63.35,12237053,62.11,61.09,68.347339,0.975000,1,103792.832092,0.011068,0.037928
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 11:30:00-04:00,132.24,132.32,130.96,131.49,5249743,133.70,136.57,28.648649,1.957143,1,179313.677126,0.005632,0.793137
2025-07-02 12:30:00-04:00,132.66,133.37,132.11,132.22,6213849,133.60,136.42,32.770745,1.904286,1,179883.658284,0.003179,0.798837
2025-07-02 13:30:00-04:00,132.51,133.30,132.18,132.67,5675254,133.50,136.26,32.002801,1.927143,1,179680.093584,-0.001132,0.796801


In [63]:
# Plot the cumulative returns of the model from the backtesting
lr_backtest_cumulative['Cumulative_Returns'].hvplot()

:Curve   [Datetime]   (Cumulative_Returns)

##### LSTM

In [64]:
# Conduct backtest with the function created
lstm_backtest = backtest_df(X_test, y_pred_lstm)

# Obtain the cumulateive returns the backtesting algorithm generates
lstm_backtest_cumulative = cumualtive_returns(lstm_backtest, X_test)
lstm_backtest_cumulative

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Model_Predictions,Portfolio_Total,Portfolio_Returns,Cumulative_Returns
Datetime,,,,,,,,,,,,,
2024-11-21 14:30:00-05:00,61.58,61.74,61.30,61.59,4064551,61.75,60.79,45.707376,1.046429,1,99800.024988,-0.002000,-0.002000
2024-11-21 15:30:00-05:00,61.35,61.72,61.24,61.57,3388214,61.71,60.81,39.641944,1.027143,1,99426.713754,-0.003741,-0.005733
2024-11-22 09:30:00-05:00,62.21,63.04,61.37,61.60,15777196,61.76,60.87,63.665595,0.925000,1,100822.573148,0.014039,0.008226
2024-11-22 10:30:00-05:00,63.34,63.47,62.14,62.21,11239611,61.91,60.97,68.870523,0.956429,1,102656.667468,0.018191,0.026567
2024-11-22 11:30:00-05:00,64.04,64.38,63.00,63.35,12237053,62.11,61.09,68.347339,0.975000,1,103792.832092,0.011068,0.037928
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 11:30:00-04:00,132.24,132.32,130.96,131.49,5249743,133.70,136.57,28.648649,1.957143,0,133696.236268,0.000000,0.336962
2025-07-02 12:30:00-04:00,132.66,133.37,132.11,132.22,6213849,133.60,136.42,32.770745,1.904286,0,133696.236268,0.000000,0.336962
2025-07-02 13:30:00-04:00,132.51,133.30,132.18,132.67,5675254,133.50,136.26,32.002801,1.927143,0,133696.236268,0.000000,0.336962


In [65]:
# Plot the cumulative returns of the model from the backtesting
lstm_backtest_cumulative['Cumulative_Returns'].hvplot()

:Curve   [Datetime]   (Cumulative_Returns)

#### Plot the entry and exit points the model generated on their respective cumulative returns

In [66]:
# function to plot entry and exit
def plot_entry_exit(df, model:str):
    # Obtain exit and entry points (1 to buy, -1 to sell and 0 to hold)
    df['Entry/Exit'] = df['Model_Predictions'].diff()
    df['Entry/Exit'][0] = df['Model_Predictions'][0]
    df['Entry/Exit'] = df['Entry/Exit'].astype(int)
    df[['Model_Predictions', 'Entry/Exit']]
    

    # Plot the points to sell on the graph
    exit = df[df['Entry/Exit'] == -1]['Portfolio_Total'].hvplot.scatter(color = 'red',
                                           marker = 'v',
                                           legend = False,
                                           width = 1000,
                                           height = 500,
                                           size = 200, ylabel = 'Price in $')

    # Plot the points to buy on the graph
    entry = df[df['Entry/Exit'] == 1]['Portfolio_Total'].hvplot.scatter(color = 'green',
                                           marker = '^',
                                           legend = False,
                                           width = 1000,
                                           height = 500,
                                           size = 200, ylabel = 'Price in $')

    


    # plot cumulative returns through the model
    portfolio_price_chart = df['Portfolio_Total'].hvplot(color = 'lightgray')

    entry_exit_chart = portfolio_price_chart * entry * exit 

    entry_exit_chart.opts(title = f'Entry Exit Plot on the Portfolio Total with the {model} model',
                         height = 500,
                         width = 1000)

    return entry_exit_chart

In [67]:
# Make the list of dataframes with the cumulative returns of each model
model_pred = [gb_backtest_cumulative, svm_backtest_cumulative, lr_backtest_cumulative, lstm_backtest_cumulative]

# List of names of the respective models
model_name = ['Gradient Boosting Classifier', 
             'Supported Vector Machine',
             'Logistic Regression Classifier',
             'LSTM']

# Loop throught the lists to pass them as arguments in to the function to generate plots
for i in range(len(model_name)):
    display(plot_entry_exit(model_pred[i], model_name[i]))

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

## Step 5: Calculating Metrics - Post trade performance analysis of the strategy and models

#### Trade-Level Performance Extraction
The performance function extracts detailed trade records from the backtesting results for a specified model.

Functionality :
Iterates over the backtest DataFrame to identify trade entry and exit points.

For each completed trade, records the following:
- Stock ticker (fixed as "PLTR")
- Entry date and price
- Exit date and price
- Number of shares traded
- Portfolio value at entry and exit
- Profit or loss from the trade

Purpose : This function allows granular analysis of individual trades, providing insights into timing, trade size, and profitability.

Output : Returns a DataFrame summarizing all executed trades with their corresponding performance metrics, aiding comprehensive evaluation of the trading model.

In [68]:
# Create a function to obtain the records of the trades with the respective models
def performance(df, model:str):
    # Create the dataframe specifying the features of the trades along with its profit and loss
    performance_data = []
    
    # Initialize entry_date as None
    entry_date = None

    for index, row in df.iterrows():
        if row['Entry/Exit'] == 1:
            entry_date = index
            entry_share_price = row['Close']
            share_size = abs(row['Portfolio_Total'] / row['Close'])
            entry_portfolio_holdings = row['Close'] * (abs(row['Portfolio_Total'] / row['Close']))

        elif row['Entry/Exit'] == -1 and entry_date is not None:
            exit_date = index
            exit_share_price = row['Close']
            share_size = abs(row['Portfolio_Total'] / row['Close'])
            exit_portfolio_holdings = row['Close'] * (abs(row['Portfolio_Total'] / row['Close']))
            profit_loss = exit_portfolio_holdings - entry_portfolio_holdings

            performance_data.append({
                'Stock': 'PLTR',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Entry Price': entry_share_price,
                'Exit Price': exit_share_price,
                'Shares': share_size,
                'Entry Portfolio Holding': entry_portfolio_holdings,
                'Exit Portfolio Holding': exit_portfolio_holdings,
                'Profit/Loss': profit_loss
            })



    performance_data_df = pd.DataFrame(performance_data)
    print(f'\n \nPerformance of {model} : ')
    
    return performance_data_df

In [69]:
# Loop through the list of model names and generate the performance of each model
for i in range(len(model_name)):
    display(performance(model_pred[i], model_name[i]))


 
Performance of Gradient Boosting Classifier : 


,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-21 14:30:00-05:00,2024-12-27 15:30:00-05:00,61.58,79.13,1619.847351,99800.024988,128178.520917,28378.495930
1,PLTR,2024-12-30 10:30:00-05:00,2024-12-30 15:30:00-05:00,78.16,77.13,1635.853772,127922.195904,126173.401459,-1748.794446
2,PLTR,2024-12-31 10:30:00-05:00,2024-12-31 11:30:00-05:00,76.81,76.42,1638.565554,125921.086183,125219.179610,-701.906574
3,PLTR,2025-01-02 10:30:00-05:00,2025-03-26 13:30:00-04:00,74.96,92.06,1666.307123,124968.772540,153400.233785,28431.461245
4,PLTR,2025-03-26 15:30:00-04:00,2025-03-27 09:30:00-04:00,92.25,92.36,1658.721299,153093.471648,153199.499190,106.027542
5,PLTR,2025-03-27 10:30:00-04:00,2025-05-12 09:30:00-04:00,92.63,115.65,1649.755011,152893.138473,190794.167046,37901.028574
6,PLTR,2025-05-12 10:30:00-04:00,2025-05-12 12:30:00-04:00,118.43,117.64,1607.004670,190412.626387,189048.029337,-1364.597050
7,PLTR,2025-05-13 10:30:00-04:00,2025-05-16 09:30:00-04:00,126.59,128.34,1489.657850,188669.980517,191182.688408,2512.707891
8,PLTR,2025-05-16 10:30:00-04:00,2025-05-19 11:30:00-04:00,128.48,125.39,1484.317511,190800.370803,186118.572751,-4681.798052
9,PLTR,2025-05-20 10:30:00-04:00,2025-05-20 11:30:00-04:00,125.85,126.21,1475.197842,185746.382112,186184.719593,438.337481



 
Performance of Supported Vector Machine : 


,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-21 14:30:00-05:00,2025-01-02 13:30:00-05:00,61.58,74.32,1619.847351,99800.024988,120387.055157,20587.030170
1,PLTR,2025-01-02 14:30:00-05:00,2025-01-07 11:30:00-05:00,74.85,70.93,1604.359762,120146.311128,113797.237907,-6349.073222
2,PLTR,2025-01-07 12:30:00-05:00,2025-01-07 13:30:00-05:00,71.23,70.97,1593.611853,113569.671866,113098.633211,-471.038655
3,PLTR,2025-01-14 10:30:00-05:00,2025-01-14 12:30:00-05:00,66.16,66.27,1705.201220,112872.464205,113003.684846,131.220641
4,PLTR,2025-01-14 14:30:00-05:00,2025-01-15 09:30:00-05:00,65.56,68.45,1719.362439,112777.705713,117690.358978,4912.653265
5,PLTR,2025-01-15 10:30:00-05:00,2025-02-20 10:30:00-05:00,67.67,100.18,1734.836240,117455.007668,173795.894498,56340.886830
6,PLTR,2025-02-20 15:30:00-05:00,2025-02-24 09:30:00-05:00,106.26,89.68,1631.486469,173448.346137,146311.706527,-27136.639610
7,PLTR,2025-03-03 10:30:00-05:00,2025-03-04 09:30:00-05:00,87.98,80.77,1658.856783,146019.119673,133985.862335,-12033.257339
8,PLTR,2025-03-04 13:30:00-05:00,2025-03-07 09:30:00-05:00,85.85,81.99,1556.798666,133717.924090,127641.922664,-6076.001426
9,PLTR,2025-03-07 10:30:00-05:00,2025-03-10 11:30:00-04:00,80.24,78.74,1586.778078,127386.670713,124942.905896,-2443.764817



 
Performance of Logistic Regression Classifier : 


,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-21 14:30:00-05:00,2025-01-07 11:30:00-05:00,61.58,70.93,1619.847351,99800.024988,114895.772636,15095.747648
1,PLTR,2025-01-07 12:30:00-05:00,2025-01-08 10:30:00-05:00,71.23,67.27,1608.995688,114666.009800,108237.139955,-6428.869845
2,PLTR,2025-01-08 14:30:00-05:00,2025-01-10 10:30:00-05:00,68.31,65.32,1580.541114,108020.692721,103240.945542,-4779.747179
3,PLTR,2025-01-10 14:30:00-05:00,2025-01-13 10:30:00-05:00,67.15,64.54,1533.626948,103034.489448,98980.283222,-4054.206226
4,PLTR,2025-01-14 10:30:00-05:00,2025-02-20 10:30:00-05:00,66.16,100.18,1492.337218,98782.347388,149502.342549,50719.995161
5,PLTR,2025-02-20 13:30:00-05:00,2025-02-24 10:30:00-05:00,103.78,93.51,1436.971339,149203.375221,134371.189898,-14832.185323
6,PLTR,2025-02-27 10:30:00-05:00,2025-02-27 15:30:00-05:00,90.14,84.77,1486.970607,134102.481094,126050.498370,-8051.982723
7,PLTR,2025-03-03 09:30:00-05:00,2025-03-04 10:30:00-05:00,88.88,80.34,1414.667238,125798.428871,113654.365888,-12144.062982
8,PLTR,2025-03-04 12:30:00-05:00,2025-03-10 14:30:00-04:00,83.14,76.12,1363.609059,113427.085556,103797.921574,-9629.163982
9,PLTR,2025-03-11 09:30:00-04:00,2025-03-31 10:30:00-04:00,75.87,82.76,1364.684780,103590.351668,112941.312397,9350.960729



 
Performance of LSTM : 


,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-21 14:30:00-05:00,2024-12-09 10:30:00-05:00,61.58,72.66,1619.847351,99800.024988,117698.108554,17898.083566
1,PLTR,2024-12-09 11:30:00-05:00,2024-12-27 13:30:00-05:00,72.54,78.79,1618.473926,117462.741746,127519.560596,10056.818849
2,PLTR,2025-01-02 10:30:00-05:00,2025-01-02 11:30:00-05:00,74.96,74.23,1696.918578,127264.553338,125962.266044,-1302.287295
3,PLTR,2025-01-03 10:30:00-05:00,2025-01-07 11:30:00-05:00,79.08,70.93,1588.867124,125710.372987,112698.345116,-13012.027871
4,PLTR,2025-01-08 10:30:00-05:00,2025-01-08 11:30:00-05:00,67.27,66.61,1671.128654,112472.976586,111313.879625,-1159.096960
5,PLTR,2025-01-10 10:30:00-05:00,2025-01-10 11:30:00-05:00,65.32,66.47,1699.874731,111091.279681,112990.673384,1899.393703
6,PLTR,2025-01-10 13:30:00-05:00,2025-01-13 13:30:00-05:00,67.43,63.90,1671.487803,112764.720271,106808.070624,-5956.649647
7,PLTR,2025-01-14 09:30:00-05:00,2025-01-27 13:30:00-05:00,67.22,74.37,1584.963760,106594.481171,117873.754841,11279.273670
8,PLTR,2025-01-27 14:30:00-05:00,2025-02-11 11:30:00-05:00,74.20,114.14,1584.626767,117638.036785,180869.299143,63231.262358
9,PLTR,2025-02-12 10:30:00-05:00,2025-02-12 11:30:00-05:00,115.29,115.76,1564.901444,180507.605740,181152.991132,645.385392


#### Performance Evaluation Metrics Calculation

- Define `sortino` function to calculate the Sortino Ratio:
  - Extract numeric daily portfolio returns, excluding missing values.
  - Compute average daily return and downside deviation (standard deviation of negative returns).
  - Calculate annualized Sortino Ratio to assess risk-adjusted returns penalizing downside volatility.

- Define `evaluation` function to compute key backtesting performance metrics:
  - Annualized Returns: average daily return scaled by trading periods (252 days).
  - Cumulative Returns: total compounded returns over the period.
  - Annualized Volatility: standard deviation of daily returns scaled annually.
  - Sharpe Ratio: risk-adjusted return using total volatility.
  - Sortino Ratio: downside risk-adjusted return calculated via the `sortino` function.
  - Return metrics in a structured DataFrame for easy comparison.

- Apply the evaluation function to cumulative returns data from each model (Gradient Boosting, SVM, Logistic Regression, LSTM).
- Consolidate all model metrics into a single DataFrame and rename columns for clarity.
- Display the comparative performance summary across all models.

This step provides quantitative insight into each model’s trading strategy performance using common financial risk and return metrics.

In [70]:
# Create a function to calculate sortino ratio
def sortino(df):
    # Convert the returns to numneric and drop 'NaN's for calulation
    daily_returns = pd.to_numeric(df['Portfolio_Returns']).dropna()


    # Calculate average daily return
    average_return = daily_returns.mean()

    # Calculate downside returns (only negative returns)
    downside_returns = daily_returns[daily_returns < 0]

    # Calculate the downside standard deviation
    downside_std = downside_returns.std()

    # Calculate the Sortino Ratio
    sortino_ratio = (average_return / downside_std) * np.sqrt(252)
    
    
    return sortino_ratio

In [71]:
# Create funciton to calculate the metrics in the dataframe format
def evaluation(df):
    
    # Create dataframe to record the perfoemance metrics
    metics = ['Annualized Returns',
              'Cumulative Returns',
              'Annualized Volatility',
              'Sharpe Ratio',
              'Sortino Ratio']

    eval_df = pd.DataFrame(columns = ['Backtest'], index = metics)
    
    eval_df.loc['Annualized Returns'] = df['Portfolio_Returns'].mean() * 252
    eval_df.loc['Cumulative Returns'] = df['Cumulative_Returns'][-1]
    eval_df.loc['Annualized Volatility'] = df['Portfolio_Returns'].std() * np.sqrt(252)
    eval_df.loc['Sharpe Ratio'] = (df['Portfolio_Returns'].mean() * 252) / (df['Portfolio_Returns'].std() * np.sqrt(252))
    eval_df.loc['Sortino Ratio'] = sortino(df)
    
    
    return eval_df

##### Gradient Boosting Classifier

In [72]:
# Generate the performance mertics of the Gradient Boosting Classifier model
gb_backtest = evaluation(gb_backtest_cumulative)
gb_backtest

,Backtest
Annualized Returns,0.189088
Cumulative Returns,0.820418
Annualized Volatility,0.300131
Sharpe Ratio,0.630018
Sortino Ratio,0.76494


##### Supported Vector Machine

In [73]:
# Generate the performance mertics of the Supported Vector Machine model
svm_backtest = evaluation(svm_backtest_cumulative)
svm_backtest

,Backtest
Annualized Returns,0.208895
Cumulative Returns,0.988296
Annualized Volatility,0.295106
Sharpe Ratio,0.707865
Sortino Ratio,0.85507


##### Logistic Regression Classifier

In [74]:
# Generate the performance mertics of the Logistic Regression Classifier model
lr_backtest = evaluation(lr_backtest_cumulative)
lr_backtest

,Backtest
Annualized Returns,0.186131
Cumulative Returns,0.79178
Annualized Volatility,0.303012
Sharpe Ratio,0.614268
Sortino Ratio,0.761678


##### LSTM

In [75]:
# Generate the performance mertics of the LSTM model
lstm_backtest = evaluation(lstm_backtest_cumulative)
lstm_backtest

,Backtest
Annualized Returns,0.10831
Cumulative Returns,0.336962
Annualized Volatility,0.278355
Sharpe Ratio,0.389107
Sortino Ratio,0.418777


In [76]:
# Concatinate the metrics of all models
consolidated_metrics_df = pd.concat([gb_backtest, svm_backtest, lr_backtest, lstm_backtest], axis = 1)

# Rename columns
consolidated_metrics_df.columns = ['Gradient Boosting', 'Support Vector Machine', 'Logistic Regression', 'LSTM']

# Display the final result
consolidated_metrics_df


,Gradient Boosting,Support Vector Machine,Logistic Regression,LSTM
Annualized Returns,0.189088,0.208895,0.186131,0.10831
Cumulative Returns,0.820418,0.988296,0.79178,0.336962
Annualized Volatility,0.300131,0.295106,0.303012,0.278355
Sharpe Ratio,0.630018,0.707865,0.614268,0.389107
Sortino Ratio,0.76494,0.85507,0.761678,0.418777


##### LSTM

In [64]:
# Conduct backtest with the function created
lstm_backtest = backtest_df(X_test, y_pred_lstm)

# Obtain the cumulateive returns the backtesting algorithm generates
lstm_backtest_cumulative = cumualtive_returns(lstm_backtest, X_test)
lstm_backtest_cumulative

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Model_Predictions,Portfolio_Total,Portfolio_Returns,Cumulative_Returns
Datetime,,,,,,,,,,,,,
2024-11-21 14:30:00-05:00,61.58,61.74,61.30,61.59,4064551,61.75,60.79,45.707376,1.046429,1,99800.024988,-0.002000,-0.002000
2024-11-21 15:30:00-05:00,61.35,61.72,61.24,61.57,3388214,61.71,60.81,39.641944,1.027143,1,99426.713754,-0.003741,-0.005733
2024-11-22 09:30:00-05:00,62.21,63.04,61.37,61.60,15777196,61.76,60.87,63.665595,0.925000,1,100822.573148,0.014039,0.008226
2024-11-22 10:30:00-05:00,63.34,63.47,62.14,62.21,11239611,61.91,60.97,68.870523,0.956429,1,102656.667468,0.018191,0.026567
2024-11-22 11:30:00-05:00,64.04,64.38,63.00,63.35,12237053,62.11,61.09,68.347339,0.975000,1,103792.832092,0.011068,0.037928
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 11:30:00-04:00,132.24,132.32,130.96,131.49,5249743,133.70,136.57,28.648649,1.957143,0,133696.236268,0.000000,0.336962
2025-07-02 12:30:00-04:00,132.66,133.37,132.11,132.22,6213849,133.60,136.42,32.770745,1.904286,0,133696.236268,0.000000,0.336962
2025-07-02 13:30:00-04:00,132.51,133.30,132.18,132.67,5675254,133.50,136.26,32.002801,1.927143,0,133696.236268,0.000000,0.336962


In [65]:
# Plot the cumulative returns of the model from the backtesting
lstm_backtest_cumulative['Cumulative_Returns'].hvplot()

:Curve   [Datetime]   (Cumulative_Returns)

#### Plot the entry and exit points the model generated on their respective cumulative returns

In [66]:
# function to plot entry and exit
def plot_entry_exit(df, model:str):
    # Obtain exit and entry points (1 to buy, -1 to sell and 0 to hold)
    df['Entry/Exit'] = df['Model_Predictions'].diff()
    df['Entry/Exit'][0] = df['Model_Predictions'][0]
    df['Entry/Exit'] = df['Entry/Exit'].astype(int)
    df[['Model_Predictions', 'Entry/Exit']]
    

    # Plot the points to sell on the graph
    exit = df[df['Entry/Exit'] == -1]['Portfolio_Total'].hvplot.scatter(color = 'red',
                                           marker = 'v',
                                           legend = False,
                                           width = 1000,
                                           height = 500,
                                           size = 200, ylabel = 'Price in $')

    # Plot the points to buy on the graph
    entry = df[df['Entry/Exit'] == 1]['Portfolio_Total'].hvplot.scatter(color = 'green',
                                           marker = '^',
                                           legend = False,
                                           width = 1000,
                                           height = 500,
                                           size = 200, ylabel = 'Price in $')

    


    # plot cumulative returns through the model
    portfolio_price_chart = df['Portfolio_Total'].hvplot(color = 'lightgray')

    entry_exit_chart = portfolio_price_chart * entry * exit 

    entry_exit_chart.opts(title = f'Entry Exit Plot on the Portfolio Total with the {model} model',
                         height = 500,
                         width = 1000)

    return entry_exit_chart

In [67]:
# Make the list of dataframes with the cumulative returns of each model
model_pred = [gb_backtest_cumulative, svm_backtest_cumulative, lr_backtest_cumulative, lstm_backtest_cumulative]

# List of names of the respective models
model_name = ['Gradient Boosting Classifier', 
             'Supported Vector Machine',
             'Logistic Regression Classifier',
             'LSTM']

# Loop throught the lists to pass them as arguments in to the function to generate plots
for i in range(len(model_name)):
    display(plot_entry_exit(model_pred[i], model_name[i]))

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

## Calculating Metrics - Post trade performance analysis of the strategy and models

#### Profit and Loss of the trades executed

In [68]:
# Create a function to obtain the records of the trades with the respective models
def performance(df, model:str):
    # Create the dataframe specifying the features of the trades along with its profit and loss
    performance_data = []
    
    # Initialize entry_date as None
    entry_date = None

    for index, row in df.iterrows():
        if row['Entry/Exit'] == 1:
            entry_date = index
            entry_share_price = row['Close']
            share_size = abs(row['Portfolio_Total'] / row['Close'])
            entry_portfolio_holdings = row['Close'] * (abs(row['Portfolio_Total'] / row['Close']))

        elif row['Entry/Exit'] == -1 and entry_date is not None:
            exit_date = index
            exit_share_price = row['Close']
            share_size = abs(row['Portfolio_Total'] / row['Close'])
            exit_portfolio_holdings = row['Close'] * (abs(row['Portfolio_Total'] / row['Close']))
            profit_loss = exit_portfolio_holdings - entry_portfolio_holdings

            performance_data.append({
                'Stock': 'PLTR',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Entry Price': entry_share_price,
                'Exit Price': exit_share_price,
                'Shares': share_size,
                'Entry Portfolio Holding': entry_portfolio_holdings,
                'Exit Portfolio Holding': exit_portfolio_holdings,
                'Profit/Loss': profit_loss
            })



    performance_data_df = pd.DataFrame(performance_data)
    print(f'\n \nPerformance of {model} : ')
    
    return performance_data_df

In [69]:
# Loop through the list of model names and generate the performance of each model
for i in range(len(model_name)):
    display(performance(model_pred[i], model_name[i]))


 
Performance of Gradient Boosting Classifier : 


,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-21 14:30:00-05:00,2024-12-27 15:30:00-05:00,61.58,79.13,1619.847351,99800.024988,128178.520917,28378.495930
1,PLTR,2024-12-30 10:30:00-05:00,2024-12-30 15:30:00-05:00,78.16,77.13,1635.853772,127922.195904,126173.401459,-1748.794446
2,PLTR,2024-12-31 10:30:00-05:00,2024-12-31 11:30:00-05:00,76.81,76.42,1638.565554,125921.086183,125219.179610,-701.906574
3,PLTR,2025-01-02 10:30:00-05:00,2025-03-26 13:30:00-04:00,74.96,92.06,1666.307123,124968.772540,153400.233785,28431.461245
4,PLTR,2025-03-26 15:30:00-04:00,2025-03-27 09:30:00-04:00,92.25,92.36,1658.721299,153093.471648,153199.499190,106.027542
5,PLTR,2025-03-27 10:30:00-04:00,2025-05-12 09:30:00-04:00,92.63,115.65,1649.755011,152893.138473,190794.167046,37901.028574
6,PLTR,2025-05-12 10:30:00-04:00,2025-05-12 12:30:00-04:00,118.43,117.64,1607.004670,190412.626387,189048.029337,-1364.597050
7,PLTR,2025-05-13 10:30:00-04:00,2025-05-16 09:30:00-04:00,126.59,128.34,1489.657850,188669.980517,191182.688408,2512.707891
8,PLTR,2025-05-16 10:30:00-04:00,2025-05-19 11:30:00-04:00,128.48,125.39,1484.317511,190800.370803,186118.572751,-4681.798052
9,PLTR,2025-05-20 10:30:00-04:00,2025-05-20 11:30:00-04:00,125.85,126.21,1475.197842,185746.382112,186184.719593,438.337481



 
Performance of Supported Vector Machine : 


,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-21 14:30:00-05:00,2025-01-02 13:30:00-05:00,61.58,74.32,1619.847351,99800.024988,120387.055157,20587.030170
1,PLTR,2025-01-02 14:30:00-05:00,2025-01-07 11:30:00-05:00,74.85,70.93,1604.359762,120146.311128,113797.237907,-6349.073222
2,PLTR,2025-01-07 12:30:00-05:00,2025-01-07 13:30:00-05:00,71.23,70.97,1593.611853,113569.671866,113098.633211,-471.038655
3,PLTR,2025-01-14 10:30:00-05:00,2025-01-14 12:30:00-05:00,66.16,66.27,1705.201220,112872.464205,113003.684846,131.220641
4,PLTR,2025-01-14 14:30:00-05:00,2025-01-15 09:30:00-05:00,65.56,68.45,1719.362439,112777.705713,117690.358978,4912.653265
5,PLTR,2025-01-15 10:30:00-05:00,2025-02-20 10:30:00-05:00,67.67,100.18,1734.836240,117455.007668,173795.894498,56340.886830
6,PLTR,2025-02-20 15:30:00-05:00,2025-02-24 09:30:00-05:00,106.26,89.68,1631.486469,173448.346137,146311.706527,-27136.639610
7,PLTR,2025-03-03 10:30:00-05:00,2025-03-04 09:30:00-05:00,87.98,80.77,1658.856783,146019.119673,133985.862335,-12033.257339
8,PLTR,2025-03-04 13:30:00-05:00,2025-03-07 09:30:00-05:00,85.85,81.99,1556.798666,133717.924090,127641.922664,-6076.001426
9,PLTR,2025-03-07 10:30:00-05:00,2025-03-10 11:30:00-04:00,80.24,78.74,1586.778078,127386.670713,124942.905896,-2443.764817



 
Performance of Logistic Regression Classifier : 


,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-21 14:30:00-05:00,2025-01-07 11:30:00-05:00,61.58,70.93,1619.847351,99800.024988,114895.772636,15095.747648
1,PLTR,2025-01-07 12:30:00-05:00,2025-01-08 10:30:00-05:00,71.23,67.27,1608.995688,114666.009800,108237.139955,-6428.869845
2,PLTR,2025-01-08 14:30:00-05:00,2025-01-10 10:30:00-05:00,68.31,65.32,1580.541114,108020.692721,103240.945542,-4779.747179
3,PLTR,2025-01-10 14:30:00-05:00,2025-01-13 10:30:00-05:00,67.15,64.54,1533.626948,103034.489448,98980.283222,-4054.206226
4,PLTR,2025-01-14 10:30:00-05:00,2025-02-20 10:30:00-05:00,66.16,100.18,1492.337218,98782.347388,149502.342549,50719.995161
5,PLTR,2025-02-20 13:30:00-05:00,2025-02-24 10:30:00-05:00,103.78,93.51,1436.971339,149203.375221,134371.189898,-14832.185323
6,PLTR,2025-02-27 10:30:00-05:00,2025-02-27 15:30:00-05:00,90.14,84.77,1486.970607,134102.481094,126050.498370,-8051.982723
7,PLTR,2025-03-03 09:30:00-05:00,2025-03-04 10:30:00-05:00,88.88,80.34,1414.667238,125798.428871,113654.365888,-12144.062982
8,PLTR,2025-03-04 12:30:00-05:00,2025-03-10 14:30:00-04:00,83.14,76.12,1363.609059,113427.085556,103797.921574,-9629.163982
9,PLTR,2025-03-11 09:30:00-04:00,2025-03-31 10:30:00-04:00,75.87,82.76,1364.684780,103590.351668,112941.312397,9350.960729



 
Performance of LSTM : 


,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-21 14:30:00-05:00,2024-12-09 10:30:00-05:00,61.58,72.66,1619.847351,99800.024988,117698.108554,17898.083566
1,PLTR,2024-12-09 11:30:00-05:00,2024-12-27 13:30:00-05:00,72.54,78.79,1618.473926,117462.741746,127519.560596,10056.818849
2,PLTR,2025-01-02 10:30:00-05:00,2025-01-02 11:30:00-05:00,74.96,74.23,1696.918578,127264.553338,125962.266044,-1302.287295
3,PLTR,2025-01-03 10:30:00-05:00,2025-01-07 11:30:00-05:00,79.08,70.93,1588.867124,125710.372987,112698.345116,-13012.027871
4,PLTR,2025-01-08 10:30:00-05:00,2025-01-08 11:30:00-05:00,67.27,66.61,1671.128654,112472.976586,111313.879625,-1159.096960
5,PLTR,2025-01-10 10:30:00-05:00,2025-01-10 11:30:00-05:00,65.32,66.47,1699.874731,111091.279681,112990.673384,1899.393703
6,PLTR,2025-01-10 13:30:00-05:00,2025-01-13 13:30:00-05:00,67.43,63.90,1671.487803,112764.720271,106808.070624,-5956.649647
7,PLTR,2025-01-14 09:30:00-05:00,2025-01-27 13:30:00-05:00,67.22,74.37,1584.963760,106594.481171,117873.754841,11279.273670
8,PLTR,2025-01-27 14:30:00-05:00,2025-02-11 11:30:00-05:00,74.20,114.14,1584.626767,117638.036785,180869.299143,63231.262358
9,PLTR,2025-02-12 10:30:00-05:00,2025-02-12 11:30:00-05:00,115.29,115.76,1564.901444,180507.605740,181152.991132,645.385392


In [70]:
# Create a function to calculate sortino ratio
def sortino(df):
    # Convert the returns to numneric and drop 'NaN's for calulation
    daily_returns = pd.to_numeric(df['Portfolio_Returns']).dropna()


    # Calculate average daily return
    average_return = daily_returns.mean()

    # Calculate downside returns (only negative returns)
    downside_returns = daily_returns[daily_returns < 0]

    # Calculate the downside standard deviation
    downside_std = downside_returns.std()

    # Calculate the Sortino Ratio
    sortino_ratio = (average_return / downside_std) * np.sqrt(252)
    
    
    return sortino_ratio

In [71]:
# Create funciton to calculate the metrics in the dataframe format
def evaluation(df):
    
    # Create dataframe to record the perfoemance metrics
    metics = ['Annualized Returns',
              'Cumulative Returns',
              'Annualized Volatility',
              'Sharpe Ratio',
              'Sortino Ratio']

    eval_df = pd.DataFrame(columns = ['Backtest'], index = metics)
    
    eval_df.loc['Annualized Returns'] = df['Portfolio_Returns'].mean() * 252
    eval_df.loc['Cumulative Returns'] = df['Cumulative_Returns'][-1]
    eval_df.loc['Annualized Volatility'] = df['Portfolio_Returns'].std() * np.sqrt(252)
    eval_df.loc['Sharpe Ratio'] = (df['Portfolio_Returns'].mean() * 252) / (df['Portfolio_Returns'].std() * np.sqrt(252))
    eval_df.loc['Sortino Ratio'] = sortino(df)
    
    
    return eval_df

##### Gradient Boosting Classifier

In [72]:
# Generate the performance mertics of the Gradient Boosting Classifier model
gb_backtest = evaluation(gb_backtest_cumulative)
gb_backtest

,Backtest
Annualized Returns,0.189088
Cumulative Returns,0.820418
Annualized Volatility,0.300131
Sharpe Ratio,0.630018
Sortino Ratio,0.76494


##### Supported Vector Machine

In [73]:
# Generate the performance mertics of the Supported Vector Machine model
svm_backtest = evaluation(svm_backtest_cumulative)
svm_backtest

,Backtest
Annualized Returns,0.208895
Cumulative Returns,0.988296
Annualized Volatility,0.295106
Sharpe Ratio,0.707865
Sortino Ratio,0.85507


##### Logistic Regression Classifier

In [74]:
# Generate the performance mertics of the Logistic Regression Classifier model
lr_backtest = evaluation(lr_backtest_cumulative)
lr_backtest

,Backtest
Annualized Returns,0.186131
Cumulative Returns,0.79178
Annualized Volatility,0.303012
Sharpe Ratio,0.614268
Sortino Ratio,0.761678


##### LSTM

In [75]:
# Generate the performance mertics of the LSTM model
lstm_backtest = evaluation(lstm_backtest_cumulative)
lstm_backtest

,Backtest
Annualized Returns,0.10831
Cumulative Returns,0.336962
Annualized Volatility,0.278355
Sharpe Ratio,0.389107
Sortino Ratio,0.418777


In [76]:
# Concatinate the metrics of all models
consolidated_metrics_df = pd.concat([gb_backtest, svm_backtest, lr_backtest, lstm_backtest], axis = 1)

# Rename columns
consolidated_metrics_df.columns = ['Gradient Boosting', 'Support Vector Machine', 'Logistic Regression', 'LSTM']

# Display the final result
consolidated_metrics_df


,Gradient Boosting,Support Vector Machine,Logistic Regression,LSTM
Annualized Returns,0.189088,0.208895,0.186131,0.10831
Cumulative Returns,0.820418,0.988296,0.79178,0.336962
Annualized Volatility,0.300131,0.295106,0.303012,0.278355
Sharpe Ratio,0.630018,0.707865,0.614268,0.389107
Sortino Ratio,0.76494,0.85507,0.761678,0.418777
